In [35]:
#first let's just import the tokenizer and import the model
from transformers import GPT2Tokenizer,GPT2LMHeadModel
tokenizer=GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
model=GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

for name, param in model.named_parameters():
    if 'transformer.h.11' not in name and 'lm_head' not in name:
        param.requires_grad = False

In [36]:
#let's load the dataset and do preprocessing('|startoftext|' and '|endoftext|') and tokenizing after that.
from datasets import load_dataset

dataset=load_dataset("json",data_files="E:/Mohit AI projects/Axolotron AI Bot/Dataset.json")
dataset = dataset["train"].train_test_split(test_size=0.1)
train_dataset = dataset["train"]
val_dataset = dataset["test"]

def text_format(example):
    return f"<|startoftext|>Prompt:{example['prompt']}\nResponse:{example['response']}<|endoftext|>"

def tokenize(example):
    tokenized = tokenizer(
        text_format(example),
        truncation=True,
        padding="max_length",
        max_length=512
    )
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "labels": tokenized["input_ids"]  # Labels are the same as input_ids for causal LM
    }

tokenized_dataset=dataset.map(tokenize, batched=False)

Map: 100%|██████████| 1/1 [00:00<00:00, 44.69 examples/s]


In [38]:
#and let's train
from transformers import TrainingArguments, Trainer

training_arguments=TrainingArguments(
    output_dir="./gpt2-chatbot",
    per_device_train_batch_size=10,
    num_train_epochs=10,
    logging_steps=10,
    save_steps=100
)

trainer=Trainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)

trainer.train()

C:\Users\mohit\AppData\Local\Temp\ipykernel_20424\1497223028.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(
e:\Mohit AI projects\Axolotron AI Bot\venv_b\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,11.792800


TrainOutput(global_step=10, training_loss=11.792774963378907, metrics={'train_runtime': 154.0748, 'train_samples_per_second': 0.584, 'train_steps_per_second': 0.065, 'total_flos': 23516282880000.0, 'train_loss': 11.792774963378907, 'epoch': 10.0})

In [39]:
#add let's save them
save_path = "E:/Mohit AI projects/Axolotron AI Bot/The Bot2"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('E:/Mohit AI projects/Axolotron AI Bot/The Bot2\\tokenizer_config.json',
 'E:/Mohit AI projects/Axolotron AI Bot/The Bot2\\special_tokens_map.json',
 'E:/Mohit AI projects/Axolotron AI Bot/The Bot2\\vocab.json',
 'E:/Mohit AI projects/Axolotron AI Bot/The Bot2\\merges.txt',
 'E:/Mohit AI projects/Axolotron AI Bot/The Bot2\\added_tokens.json')

In [40]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained(save_path)
tokenizer = GPT2Tokenizer.from_pretrained(save_path)

In [41]:
def generate_response(prompt):
    input_text=f"<|startoftext|>Prompt: {prompt}\nResponse:"
    inputs=tokenizer(input_text,return_tensors='pt')
    outputs=model.generate(
    inputs['input_ids'], 
    attention_mask=inputs.attention_mask, 
    max_length=50,          # increase max tokens to generate
    pad_token_id=tokenizer.eos_token_id,  # avoid warnings
    do_sample=True,        # to add randomness, otherwise greedy decoding might generate empty
    top_k=50,
    top_p=0.95,
    num_return_sequences=1
)
    decoded=tokenizer.decode(outputs[0])
    return decoded.strip()

generate_response("how can i contact you?")

'<|startoftext|>Prompt: how can i contact you?\nResponse: no information has been posted. [link]\n1:54:22 PM\nMessage in [main:main] (click to show/hide dialog'

In [42]:
def generate_response(prompt):
    input_text = f"<|startoftext|>Prompt: {prompt}\nResponse:"  # note colon after Response
    inputs = tokenizer(input_text, return_tensors='pt')
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=150,  # increased max length
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1,
        early_stopping=True
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the answer part after "Response:"
    if "Response:" in decoded:
        return decoded.split("Response:")[-1].strip()
    else:
        return decoded.strip()

generate_response("how can i contact you?")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


'<input type="text" name="Address" value="<|>The email address for your'